In [47]:
from utils import get_credentials
key_path = '../google-cloud-project-keys/bubbly-area-346718.json'
PROJECT_ID = 'bubbly-area-346718'
REGION = 'us-central1'
credentials = get_credentials(key_path)

In [48]:
import vertexai

vertexai.init(project= PROJECT_ID, location= REGION, credentials= credentials)

In [49]:
import pandas as pd

In [50]:
so_database = pd.read_csv('so_database_app.csv')

In [51]:
print('shape' + str(so_database.shape))
print(so_database)

shape(2000, 3)
                                             input_text  \
0     python's inspect.getfile returns "<string>"<p>...   
1     Passing parameter to function while multithrea...   
2     How do we test a specific method written in a ...   
3     how can i remove the black bg color of an imag...   
4     How to extract each sheet within an Excel file...   
...                                                 ...   
1995  Is it possible to made inline-block elements l...   
1996  Flip Clock code works on Codepen and doesn't w...   
1997  React Native How can I put one view in front o...   
1998  setting fixed width with 100% height of the pa...   
1999  How to make sidebar button not bring viewpoint...   

                                            output_text category  
0     <p><code>&lt;string&gt;</code> means that the ...   python  
1     <p>Try this and note the difference:</p>\n<pre...   python  
2     <p>Duplicate of <a href="https://stackoverflow...   python  
3     <p

In [52]:
from vertexai.language_models import TextEmbeddingModel


In [53]:
embedding_model = TextEmbeddingModel.from_pretrained('textembedding-gecko@001')

In [54]:
import pickle
with open('question_embeddings_app.pkl', 'rb') as file:
    question_embeddings = pickle.load(file)

    print(question_embeddings)

[[-0.03571156 -0.00240684  0.05860338 ... -0.03100227 -0.00855574
  -0.01997405]
 [-0.02024316 -0.0026255   0.01940405 ... -0.02158143 -0.05655403
  -0.01040497]
 [-0.05175979 -0.03712264  0.02699278 ... -0.07055898 -0.0402537
   0.00092099]
 ...
 [-0.00580394 -0.01621097  0.05829635 ... -0.03350992 -0.05343556
  -0.06016821]
 [-0.00436622 -0.02692963  0.03363771 ... -0.01686567 -0.03812337
  -0.02329491]
 [-0.04240424 -0.01633749  0.05516777 ... -0.02697376 -0.01751165
  -0.04558187]]


In [55]:
so_database['embeddings'] = question_embeddings.tolist()

In [56]:
so_database

,input_text,output_text,category,embeddings
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python,"[-0.03571155667304993, -0.0024068362545222044,..."
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python,"[-0.020243162289261818, -0.002625499852001667,..."
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python,"[-0.05175979062914848, -0.03712264448404312, 0..."
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python,"[0.02206624671816826, -0.028208276256918907, 0..."
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python,"[-0.05498068407177925, -0.0032414537854492664,..."
...,...,...,...,...
1995,Is it possible to made inline-block elements l...,<p>If this is only for the visual purpose then...,css,"[-0.009190441109240055, -0.01732615754008293, ..."
1996,Flip Clock code works on Codepen and doesn't w...,<p>You forgot to attach the CSS file for the f...,css,"[-0.009033069014549255, -0.0009270847076550126..."
1997,React Native How can I put one view in front o...,<p>You can do it using zIndex for example:</p>...,css,"[-0.005803938489407301, -0.016210969537496567,..."
1998,setting fixed width with 100% height of the pa...,<p>You can use <code>width: calc(100% - 100px)...,css,"[-0.004366223234683275, -0.02692963369190693, ..."


In [57]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

In [58]:
query = ['How to concat dataframes pandas']

In [59]:
query_embedding = embedding_model.get_embeddings(query)[0].values

In [60]:
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))

In [61]:
cos_sim_array.shape

(1, 2000)

In [62]:
index_doc_cosine = np.argmax(cos_sim_array)

In [63]:
index_doc_distances = distances_argmin([query_embedding], list(so_database.embeddings.values))

In [64]:
so_database.input_text[index_doc_distances]

1143    Add a column name to a panda dataframe (multi ...
Name: input_text, dtype: object

In [65]:
so_database.head()


,input_text,output_text,category,embeddings
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python,"[-0.03571155667304993, -0.0024068362545222044,..."
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python,"[-0.020243162289261818, -0.002625499852001667,..."
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python,"[-0.05175979062914848, -0.03712264448404312, 0..."
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python,"[0.02206624671816826, -0.028208276256918907, 0..."
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python,"[-0.05498068407177925, -0.0032414537854492664,..."


In [66]:
so_database.input_text[index_doc_distances]

1143    Add a column name to a panda dataframe (multi ...
Name: input_text, dtype: object

In [67]:
index_doc_distances

array([1143])

In [68]:
so_database.input_text[index_doc_cosine]

'Add a column name to a panda dataframe (multi index)<p>I concatenate series objects, with existing column names together to a DataFrame in Pandas. The result looks like this:</p>\n<pre><code>pd.concat([x, y, z], axis=1)\n\n\n   X   |  Y   |   Z\n  -------------------\n  data | data | data\n</code></pre>\n<p>Now I want to insert another column name A above the column names X, Y, Z, for the whole DataFrame. This should look like this at the end:</p>\n<pre><code>   A                  # New Column Name\n  ------------------- \n   X   |  Y   |   Z   # Old Column Names\n  -------------------\n  data | data | data \n</code></pre>\n<p>So far I did not find a solution how to insert a column name A above the existing columns names X, Y, Z for the complete DataFrame. I would be grateful for any help. :)</p>'

In [69]:
so_database.output_text[index_doc_cosine]

"<p>Let's try with <code>MultiIndex.from_product</code> to create <code>MultiIndex</code> columns:</p>\n<pre><code>df = pd.concat([x, y, z], axis=1)\ndf.columns = pd.MultiIndex.from_product([['A'], df.columns])\n</code></pre>\n<hr />\n<pre><code>A            \nX     Y     Z\ndata  data  data\n</code></pre>"

In [70]:
from vertexai.language_models import TextGenerationModel

In [71]:
generation_model = TextGenerationModel.from_pretrained('text-bison@001')

In [72]:
context = 'Question: ' + so_database.input_text[index_doc_cosine] + "\n Answer: " + so_database.output_text[index_doc_cosine]

In [73]:
index_doc_cosine

np.int64(1143)

In [74]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [75]:
from IPython.display import Markdown, display
t_value = 0.2
response = generation_model.predict(prompt = prompt, temperature = t_value, max_output_tokens =1024)
display(Markdown(response.text))

To concatenate dataframes in pandas, you can use the <code>concat()</code> function. The <code>concat()</code> function takes a list of dataframes as input and concatenates them along the specified axis. The default axis is 0, which means that the dataframes will be concatenated along the rows. To concatenate dataframes along the columns, you can specify the <code>axis</code> argument to be 1.

Here is an example of how to concatenate two dataframes in pandas:

```
import pandas as pd

df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'C': [7, 8, 9], 'D': [10, 11, 12]})

df = pd.concat([df1, df2], axis=1)

print(df)
```

This will output the following dataframe:

```
   A  B  C  D
0  1  4  7 10
1  2  5  8 11
2  3  6  9 12
```

In [76]:
# when document does not provide useful information

In [77]:
query = ['How to make perfect lasagna']

In [78]:
query_embedding = embedding_model.get_embeddings(query)[0].values

In [79]:
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
cos_sim_array

array([[0.54534061, 0.49939449, 0.52136036, ..., 0.46877691, 0.55854909,
        0.49552776]])

In [80]:
index_doc = np.argmax(cos_sim_array)

In [81]:
context = 'Question: ' + so_database.input_text[index_doc] + "\n Answer: " + so_database.output_text[index_doc]

In [82]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, answer with 
             [I couldn't find a good match in the \
             document database for your query]
             """

In [83]:
t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)
display(Markdown(response.text))

I couldn't find a good match in the document database for your query